# 练习：统计显著性

假设我们为一项网络实验收集了数据。在实验中，我们检验了产品信息页的布局变化，看看这种变化是否会影响到点击按钮并转到下载页的用户所占的比例。该实验以 Cookie 为分组依据，我们记录了用户的两项数据：他们访问的是哪个网页版本，以及在数据记录阶段是否访问了下载页。（在此示例中，我们没有跟踪任何其他因素，例如页面查看次数，或从访问网页到下载产品的时间间隔，这些因素可能值得进一步研究。）

在此 notebook 中，你的目标是对这两个记录指标执行统计学检验，看看两组之间是否存在统计学差异。

In [1]:
# import packages

import numpy as np
import pandas as pd
import scipy.stats as stats
from statsmodels.stats import proportion as proptests

import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
# import data

data = pd.read_csv('data/statistical_significance_data.csv')
data.head(10)

,condition,click
0,1,0
1,0,0
2,0,0
3,1,1
4,1,0
5,1,0
6,0,0
7,1,1
8,0,0
9,1,0


在数据集的“condition”列中，0 表示对照组，1 表示实验组。对于“click”列，0 表示没有点击，1 表示点击了。

## 检查不变指标

首先，我们应该检查分配给每组的用户人数是否差不多。务必要检查作为前提条件的不变指标，这样可以保证根据评估指标做出的推断有扎实的依据。如果我们发现两组在不变指标方面不平衡，则需要仔细观察访问者的划分方式，看看能否找出任何偏差原因。如果发现不变指标存在统计学显著性差异，则可能需要修改随机分配流程，并重新收集数据。

在这种情形下，我们需要对分配给某个条件的访问者比例进行双边假设检验。选择对照组还是实验组并不重要：你会获得相同的结果。你可以选择使用任何一种方法，我们将在下面主要介绍两种方法。

如果你想采用模拟方法，你可以模拟分配到每组的访问者人数，假设按照 50/50 的比例划分。重复这一过程很多次（在此示例中，重复 200 000 次应该会达到很好的速度可变性平衡），然后看看有多少模拟情形完全偏离了 50/50 的分配结果。注意，由于我们完成的是双边检验，因此极端情况还包括 50/50 相反情形的值。（例如，由于 0.48 及更低的模拟结果被视为比 0.48 的实际观测值更极端，所以 0.52 及更高的模拟结果也一样。）我们可以根据被标记模拟结果的比例获得 p 值，并用p 值评估观察到的比例。我们希望看到更大的 p 值，表明拒绝零假设的证据不足。

如果你想采取分析方法，可以使用精确二项分布计算检验的 p 值。但是，更常见的方法是使用正态分布逼近法。由于样本量很大，并且存在中心极限定理，因此这种逼近法是可行的。要获得精确的 p 值，还应该进行连续校正， 
在计算曲线下面积之前，使总数加上或减去 0.5。（例如，如果对照组的分配比例是 415 / 850，那么正态逼近计算的面积向左为 $(415 + 0.5) / 850 = 0.489$，向右为 $(435 - 0.5) / 850 = 0.511$。）

完成下面的 workspace 后，你可以查看下个页面的解答内容。你还可以尝试多种方法，看看它们是否会取得相似的结果。

### 分析方法

构建检验统计量:
$$n_{control} + 0.5$$
检验统计量的均值和方差分别为：
$$\mu = n_{obs} * p$$
$$\sigma^2 = n_obs * p * (1-p)$$
检验统计量服从二项分布$b(n_{obs}, p)$，因此可以构建服从标准正态分布的检验分布量：
$$Z=\frac{n_{control} + 0.5 - \mu}{\sigma}$$

In [3]:
# 获取实验次数和对照组次数
n_obs = data.shape[0]
n_control = data.groupby('condition').size()[0]

In [4]:
# 计算z-score
p = 0.5
sd = np.sqrt(n_obs * p * (1-p))
z = ((n_control + 0.5) - n_obs * p) / sd
print(z)

-0.506217597735


In [5]:
# 计算p-value，双边检验
print(2 * stats.norm.cdf(z))

0.612703902554


### 模拟方法

通过200000次概率为0.5的随机实验来模拟抽取对照组和实验组的过程，然后计算对照组数据大于$n_{control}$或小于$n_{obs}-n_{control}$的概率

In [6]:
# 获取实验次数和对照组次数
n_obs = data.shape[0]
n_control = data.groupby('condition').size()[0]

In [7]:
# 获取200000次实验的样本
p = 0.5
n_trials = 2e5

samples = np.random.binomial(999, p, int(n_trials))

In [8]:
# 计算p-value
print(np.logical_or(samples <= n_control, samples >= (n_obs - n_control)).mean())

0.61256


## 检查评估指标

检查完不变指标后，我们可以继续对评估指标进行假设检验，即点击率。在此示例中，我们希望看到实验组的点击率比对照组的高很多，这是一个单尾检验。

对于该指标来说，模拟方法与不变指标的区别不大。你需要将总体点击率作为共同比例，并从中为每个小组抽取模拟值。此外，需要模拟更多次，因为该检验的方差更高。

有几种分析方法可以采用，但是你很可能会再次利用正态逼近法。除了合并点击率之外，还需要计算合并标准差，以便计算 z 分数。虽然也可以采用连续校正，但比模拟通常暗示的 p 值更保守。在不连续校正的情况下计算 z 分数和 p 值应该更接近模拟结果，但是更加确信两组之间存在统计学差异。

与上个问题一样，你可以在 workspace 后面找到练习答案。

### 分析方法

In [11]:
# 计算不同condition下的点击概率
p_click = data.groupby('condition').mean()['click']
p_click

condition
0    0.079430
1    0.112205
Name: click, dtype: float64

In [18]:
# 获得实验组数量和对照组数量
n_control = data.groupby('condition').size()[0]
n_trial = data.groupby('condition').size()[1]
# 计算所有实验数据的点击概率
p_null = data['click'].mean()

In [22]:
# 计算标准差、z-score
se_p = np.sqrt(p_null * (1 - p_null) * (1/n_control + 1/n_trial))
z = (p_click[1] - p_click[0])/se_p

In [24]:
# 计算p-value
print(z)
print(1 - stats.norm.cdf(z))

1.75718873962
0.0394428219746


### 模拟方法

In [27]:
# 获得实验组数量和对照组数量
n_control = data.groupby('condition').size()[0]
n_exper = data.groupby('condition').size()[1]
# 计算所有实验数据的点击概率
p_null = data['click'].mean()

In [28]:
# 在所有实验数据的点击概率下，重复实验20000次
n_trials = 200000

ctrl_clicks = np.random.binomial(n_control, p_null, n_trials)
exper_clicks = np.random.binomial(n_exper, p_null, n_trials)
samples = exper_clicks/n_exper - ctrl_clicks/n_control

print((samples >= (p_click[1] - p_click[0])).mean())

0.039245
